This notebook to create a overall system, include Unsupervised(Isolation Forest, XGBOD) at last this notebook.

+ Choose dataset ```dataset_type='SoundDr'```:
+ Choose pretrain model to extract Feature ```PRETRAIN="FRILL"```
+ Choose model to classify ```Classifier="SVM"```
+ Set seed ```seed="2022"```

# Library

In [1]:
import warnings
from glob import glob

warnings.filterwarnings("ignore")

import os, time, math, random, cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile, pickle, h5py, joblib, json
import multiprocessing

import librosa
import opensmile
import xgboost as xgb
# import tensorflow_hub as hub

from math import pi
from tqdm import tqdm
from pathlib import Path
from functools import partial
from scipy.fftpack import fft, hilbert
from scipy.spatial.distance import cdist
from scipy.spatial import cKDTree
from sklearn.svm import LinearSVC
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, auc, precision_recall_curve, roc_curve, average_precision_score

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


# Configuration

+ Classifier: Model need to classify. Currently, We support SVM and XGBoost Classify. We show Unsupervised(Isolation Forest, XGBOD) at last this notebook.
+ PRETRAIN: Feature type need to extract for classify model. We support these: TRILL, FRILL, OpenSmile, OpenSmileBoAW, DeepSpectrum
+ dataset_type: dataset type which we need run on such as SoundDr, CoughVid, Coswara

In [2]:
Classifier = 'SVM'
PRETRAIN = 'FRILL'  # TRILL, FRILL, OpenSmile, OpenSmileBoAW, DeepSpectrum
dataset_type = 'SoundDr'

#sample rate
SR = 44100
#100 ms
FRAME_LEN = int(SR / 10)
#50% overlap, meaning 5ms hop length
HOP = int(FRAME_LEN / 2)
#the MFCC dimension
MFCC_dim = 13
codebook_size = 1000
fold_num = 5
seed = 2022

if dataset_type == 'SoundDr':
    PERIOD = 15
    SR = 48000
    DIR_DATA = "./sounddr_data/"

    df = pd.read_csv(DIR_DATA + 'data.csv')
    df['file_path'] = df['file_cough'] + '.wav'

    df['label_symptom'] = (df['symptoms_status_choice'].map(str) != "['No']").astype(int)
    df['label_abnormal'] = ((df['symptoms_status_choice'].map(str) != "['No']") | (df['cov19_status_choice'] != 'never')).astype(int)
    df['label_covid'] = (df['cov19_status_choice'] != 'never').astype(int)
elif dataset_type == 'CoughVid':
    PERIOD = 10
    SR = 22050

    DIR_DATA = './coughvid_data/'

    VidData   = pd.read_csv(os.path.join(DIR_DATA, 'public_dataset/metadata_compiled.csv'), header=0)
    VidData   = VidData.loc[VidData['cough_detected'] >= 0.9][['uuid','fever_muscle_pain','respiratory_condition','status', 'quality_1', 'age', 'gender']]
    VidData.dropna(subset=['uuid','fever_muscle_pain','respiratory_condition','status'], inplace=True)
    VidData = VidData[(VidData['quality_1'] != 'no_cough') & (VidData['quality_1'] != 'poor')]
    VidData = VidData[(VidData['status'] != 'symptomatic') & (VidData['status'].notna())]
    VidData['label_covid'] = (VidData['status'] == 'COVID-19').astype(int)

    extradata = VidData.loc[VidData['status']=='COVID-19']
    notradata = VidData.loc[VidData['status']!='COVID-19']

    df = pd.concat([extradata, notradata], ignore_index= True)
    df['file_path'] = df['uuid'].apply(lambda x: 'public_dataset/' + x + '.webm')
    def g(x):
        for i in x:
            if i is True:
                return 1
        return 0
    df['label_abnormal'] = df[['fever_muscle_pain', 'respiratory_condition', 'label_covid']].apply(lambda x: g(x), axis=1)
else:
    PERIOD = 5
    SR = 44100
    DIR_DATA = "./coswara_data/"

    join_by = pd.read_csv(os.path.join(DIR_DATA, 'combined_data.csv'))
    df_list = []
    for each in os.listdir(DIR_DATA):
        for path in tqdm(glob(DIR_DATA + each + '/*/cough-shallow.wav')):
            temp = pd.DataFrame(columns=['id', 'DIR'])
            temp['id'] = [path.split('/')[-2]]
            temp['DIR'] = [path]
            temp = pd.merge(left=temp,right=join_by,on='id',how='inner')

            temp['label_cough'] = (temp['cough'] == True).astype(int)

            temp['file_path'] = each + '/' + temp['id'] + '/cough-shallow.wav'
            temp['label_covid'] = temp['covid_status'].apply(lambda x: 1 if x == 'positive_mild' or x =='positive_moderate' or x == 'COVID-19' else 0)
            df_list.append(temp)
    df = pd.concat(df_list)
    def g(x):
        for i in x:
            if i is True:
                return 1
        return 0
    df['label_abnormal'] = df[['st', 'bd', 'cld', 'pneumonia', 'others_resp', 'asthma', 'label_covid']].apply(lambda x: g(x), axis=1)

target_col = 'label_abnormal'
OUTPUT_DIR = DIR_DATA + 'output/'
os.makedirs(OUTPUT_DIR, exist_ok = True)

df.tail()

,sex_choice,age_choice,current_city,symptoms_status_choice,medical_condition_choice,insomnia_status_choice,smoke_status_choice,cov19_status_choice,hospital_choice,cough_noise,device_model,file_cough,label,cough_duration,nose_duration,mouth_duration,file_path,label_symptom,label_abnormal,label_covid
1305,Female,20,thanh hóa,['No'],['No'],No,never,never,No,True,OPPO CPH1933,cough/good_cough_2021-08-15T13:43:33.132Z,0,25.856000,17.664000,22.357333,cough/good_cough_2021-08-15T13:43:33.132Z.wav,0,0,0
1306,Male,32,Ho Chi Minh,"['fever', 'headache']",['No'],No,never,last14,No,True,Laptop/Desktop,cough/bad_cough_2021-09-16T07:18:48.594Z,1,29.350042,29.814438,29.814438,cough/bad_cough_2021-09-16T07:18:48.594Z.wav,1,1,1
1307,Male,23,Ho Chi Minh,"['fever', 'chills', 'sorethroat', 'drycough', ...",['No'],1,1to10,last14,No,True,Laptop/Desktop,cough/bad_cough_2021-09-06T11:04:49.842Z,1,18.432000,17.152000,15.530667,cough/bad_cough_2021-09-06T11:04:49.842Z.wav,1,1,1
1308,Male,18,Ho Chi Minh,"['wetcough', 'sorethroat']",['No'],No,never,never,NaN,True,Laptop/Desktop,cough/bad_cough_2021-08-24T08:08:28.798Z,1,17.322667,17.749333,18.517333,cough/bad_cough_2021-08-24T08:08:28.798Z.wav,1,1,0
1309,Female,28,Ho Chi Minh,['No'],['No'],No,never,never,No,True,iPhone 8,cough/good_cough_2021-09-24T06:33:54.423Z,0,21.504000,26.624000,25.600000,cough/good_cough_2021-09-24T06:33:54.423Z.wav,0,0,0


In [3]:
def get_duration(filename, mono=True, res_type="kaiser_fast"):
    duration = 0
    sr = SR
    try:
        y, sr = librosa.load(filename, sr=None, mono=mono, res_type=res_type)
        duration = librosa.get_duration(y=y, sr=sr)
    except:
        print('Error file:' + filename)
    return duration, sr

In [13]:
df[target_col].value_counts()

0    825
1    485
Name: label_abnormal, dtype: int64

# Utils

In [14]:
def crop_or_pad(y, length):
    """
    Crop audio random by length
    """
    if len(y) < length:
        y = np.concatenate([y, np.zeros(length-len(y))])
    elif len(y) > length:
        cut = random.randint(0, len(y) - length)
        y = y[cut:cut+length]
    return y

def merge_feature(list_features):
    """
      Merge numpy array features
      Args:
        - list_features: list of numpy array features
                         :type: a list of numpy arrays 
      Returns:
        - features: the concatenate numpy array along axis=1
                    :type: a numpy array                 
    """      
    features = np.concatenate(list_features, axis=1)
    features = np.nan_to_num(features)
    features = np.clip(features, -np.finfo(np.float32).max, np.finfo(np.float32).max)
    return features

In [15]:
def compute_metrics(cfs_matrix):
    """
      Calculate common metrics based on the confusion matrix
      Args:
        - cfs_matrix: a sklearn confusion matrix 
                      :type: a ndarray of shape (n_classes, n_classes)
      Returns:
        - precision: the precision of the prediction
                     :type: float  
        - recall: the recall of the prediction
                  :type: float  
        - f1: the f1-score of the prediction
              :type: float                       
    """     
    precision = cfs_matrix[1,1] / (cfs_matrix[1,1] + cfs_matrix[0,1])
    recall = cfs_matrix[1,1] / (cfs_matrix[1,1] + cfs_matrix[1,0])
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1

# Extract Audio Features

Functions to extract feature follow by feature type in **Configuration** Cell(PRETRAIN) and save to cache file
If exist file, we pass over it for save time

In [16]:
if not os.path.exists(OUTPUT_DIR + PRETRAIN + '.pickle'):
    if PRETRAIN == 'TRILL':
        import tensorflow.compat.v2 as tf
        tf.enable_v2_behavior()
        import tensorflow_hub as hub
        import time
        trill_model = hub.load('https://tfhub.dev/google/nonsemantic-speech-benchmark/trill/3')

        def make_nonsemantic_trill_feat(filename):
            try:
                waveform, _ = librosa.load(os.path.join(DIR_DATA, filename), sr=16000, mono=True, res_type="kaiser_fast")
                if 2048 > waveform.shape[-1]:
                    print('File length < 2048')
                    return None, filename
                embeddings = trill_model(samples=waveform, sample_rate=16000)['embedding']
                mean_emb = embeddings.numpy().mean(axis=0)
                std_emb = embeddings.numpy().std(axis=0)
            except Exception as e:
                print('Error: ' + str(e))
                return None, filename
            return np.concatenate((mean_emb, std_emb)), filename
    elif PRETRAIN == 'FRILL':
        import tensorflow.compat.v2 as tf
        tf.enable_v2_behavior()
        import tensorflow_hub as hub

        frill_nofrontend_model = hub.load('https://tfhub.dev/google/nonsemantic-speech-benchmark/frill-nofrontend/1')

        def stabilized_log(data, additive_offset, floor):
          """TF version of mfcc_mel.StabilizedLog."""
          return tf.math.log(tf.math.maximum(data, floor) + additive_offset)


        def log_mel_spectrogram(data,
                                audio_sample_rate,
                                num_mel_bins=64,
                                log_additive_offset=0.001,
                                log_floor=1e-12,
                                window_length_secs=0.025,
                                hop_length_secs=0.010,
                                fft_length=None):
            """TF version of mfcc_mel.LogMelSpectrogram."""
            window_length_samples = int(round(audio_sample_rate * window_length_secs))
            hop_length_samples = int(round(audio_sample_rate * hop_length_secs))
            if not fft_length:
                fft_length = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))

            spectrogram = tf.abs(
                tf.signal.stft(
                    tf.cast(data, tf.dtypes.float64),
                    frame_length=window_length_samples,
                    frame_step=hop_length_samples,
                    fft_length=fft_length,
                    window_fn=tf.signal.hann_window,
                )
            )

            to_mel = tf.signal.linear_to_mel_weight_matrix(
                num_mel_bins=num_mel_bins,
                num_spectrogram_bins=fft_length // 2 + 1,
                sample_rate=audio_sample_rate,
                lower_edge_hertz=125.0,
                upper_edge_hertz=7500.0,
                dtype=tf.dtypes.float64
            )

            mel = spectrogram @ to_mel
            log_mel = stabilized_log(mel, log_additive_offset, log_floor)
            return log_mel

        def compute_frontend_features(samples, sr, frame_hop, n_required=16000, num_mel_bins=64, frame_width=96):
            if samples.dtype == np.int16:
                samples = tf.cast(samples, np.float32) / np.iinfo(np.int16).max
            if samples.dtype == np.float64:
                samples = tf.cast(samples, np.float32)
            assert samples.dtype == np.float32, samples.dtype
            n = tf.size(samples)
            samples = tf.cond(
                n < n_required,
                lambda: tf.pad(samples, [(0, n_required - n)]),
                lambda: samples
            )
            mel = log_mel_spectrogram(samples, sr, num_mel_bins=num_mel_bins)
            mel = tf.signal.frame(mel, frame_length=frame_width, frame_step=frame_hop, axis=0)
            return mel

        def make_nonsemantic_frill_nofrontend_feat(filename):
            try:
                waveform, _ = librosa.load(os.path.join(DIR_DATA, filename), sr=16000, mono=True, res_type="kaiser_fast")
                if 2048 > waveform.shape[-1]:
                    print('File length < 2048')
                    return None, filename
                frontend_feats = tf.expand_dims(compute_frontend_features(waveform, 16000, frame_hop=17), axis=-1).numpy().astype(np.float32)
                assert frontend_feats.shape[1:] == (96, 64, 1)

                embeddings = frill_nofrontend_model(frontend_feats)['embedding']
                mean_emb = embeddings.numpy().mean(axis=0)
                std_emb = embeddings.numpy().std(axis=0)
            except Exception as e:
                print('Error: ' + str(e))
                return None, filename
            return np.concatenate((mean_emb, std_emb)), filename
    elif PRETRAIN == 'OpenSmile':
        SMILE = opensmile.Smile(feature_set=opensmile.FeatureSet.ComParE_2016, feature_level=opensmile.FeatureLevel.Functionals)

        def make_opensmile_feat(filename):
            try:
                if dataset_type == 'CoughVid':
                    # convert audio to wav file
                    # for i in public_dataset/*.webm; do ffmpeg -i "$i" -c:a pcm_f32le "wav_dataset/${i%.*}.wav"; done
                    path = os.path.join(DIR_DATA, 'wav_dataset', filename[:-5]+'.wav')
                else:
                    path = os.path.join(DIR_DATA, filename)
                waveform, _ = librosa.load(path, sr=SR, mono=True, res_type="kaiser_fast")
                if 2048 > waveform.shape[-1]:
                    print('File length < 2048')
                feature = SMILE.process_signal(waveform, SR).values.reshape(-1)  # process_file, process_signal
            except Exception as e:
                print('Error: ' + str(e))
                return None, filename
            return feature, filename
    elif PRETRAIN == 'OpenSmileBoAW':
        lld = opensmile.Smile(feature_set=opensmile.FeatureSet.ComParE_2016, feature_level='lld')
        lld_deltas = opensmile.Smile(feature_set=opensmile.FeatureSet.ComParE_2016, feature_level='lld_de')

        def make_opensmileboaw_feat(filename):
            try:
                if dataset_type == 'CoughVid':
                    # convert audio to wav file
                    # for i in public_dataset/*.webm; do ffmpeg -i "$i" -c:a pcm_f32le "wav_dataset/${i%.*}.wav"; done
                    path = os.path.join(DIR_DATA, 'wav_dataset', filename[:-5]+'.wav')
                else:
                    path = os.path.join(DIR_DATA, filename)
                waveform, _ = librosa.load(path, sr=SR, mono=True, res_type="kaiser_fast")
                if 2048 > waveform.shape[-1]:
                    print('File length < 2048')
                # shape of feature 2991*65 + 2993*65 = 194610
                waveform = crop_or_pad(waveform, SR*PERIOD)
                feature = np.concatenate([
                    lld.process_signal(waveform, SR).values.reshape(-1),
                    lld_deltas.process_signal(waveform, SR).values.reshape(-1)
                ], axis=0)
            except Exception as e:
                print('Error: ' + str(e))
                return None, filename
            return feature, filename
    elif PRETRAIN == 'DeepSpectrum':
        import tensorflow as tf
        
        base_model = tf.keras.applications.densenet.DenseNet121(weights='imagenet')
        layer = base_model.layers[-2].name
        outputs = (base_model.get_layer(layer) if not hasattr(base_model.get_layer(layer), "output") else base_model.get_layer(layer).output)
        deep_model = tf.keras.models.Model(inputs=base_model.input, outputs=outputs)

        def make_deepspect_feat(filename):
            try:
                waveform, _ = librosa.load(os.path.join(DIR_DATA, filename), sr=SR, mono=True, res_type="kaiser_fast")
                if 2048 > waveform.shape[-1]:
                    print('File length < 2048')
                    return None, filename
                waveform = crop_or_pad(waveform, SR*PERIOD)
                melspec = librosa.feature.melspectrogram(waveform, sr=SR, n_mels=128, n_fft=int(0.032*SR), hop_length=int(0.016*SR))
                melspec = cv2.resize(melspec, (224, 224))
                image = np.stack([melspec, melspec, melspec], axis=-1)
                image = tf.keras.applications.densenet.preprocess_input(image)
                feature = deep_model.predict(np.array([image]))[0]
                return feature, filename
            except Exception as e:
                print('Error: ' + str(e))
                return None, filename
            return feature, filename

# Extract Features

In [17]:
def get_features_of_list_audio(df):
    X_features = []
    df['error'] = 0
    for idx, r in tqdm(df.iterrows(), total=len(df)):
        if PRETRAIN == 'TRILL':
            feature, filename = make_nonsemantic_trill_feat(r['file_path'])
        elif PRETRAIN == 'OpenSmileBoAW':
            feature, filename = make_opensmileboaw_feat(r['file_path'])
        elif PRETRAIN == 'OpenSmile':
            feature, filename = make_opensmile_feat(r['file_path'])
        elif PRETRAIN == 'DeepSpectrum':
            feature, filename = make_deepspect_feat(r['file_path'])
        else:
            feature, filename = make_nonsemantic_frill_nofrontend_feat(r['file_path'])
        
        if feature is None:
            df['error'][df['file_path'] == filename] = 1
        else:
            X_features.append(feature)
    return np.array(X_features), df

Load feature

In [18]:
if not os.path.exists(OUTPUT_DIR + PRETRAIN + '.pickle'):
    X_features, df = get_features_of_list_audio(df)
    df.to_csv(os.path.join(OUTPUT_DIR, 'data.csv'), index=False)
    pickle.dump({
        'X_trill_features': X_features
    }, open(OUTPUT_DIR + PRETRAIN + '.pickle', "wb" ))
else:
    df = pd.read_csv(os.path.join(OUTPUT_DIR, 'data.csv'))
    f = pickle.load(open(OUTPUT_DIR + PRETRAIN + '.pickle', "rb" ))
    X_features = f['X_trill_features']
df = df[df['error'] == 0].reset_index(drop=True)

# if PRETRAIN == 'OpenSmile':
scaler = StandardScaler()
X = scaler.fit_transform(merge_feature([X_features]))
# else:
# X = merge_feature([X_features])
print(f"Data feature shape: {X.shape, len(df)}")

Data feature shape: ((1310, 4096), 1310)


# Functions

In [19]:
# calculate TF
def compute_TF(word_dict, bow):
    tf_dict = {}
    bow_count = len(bow)
    for word, count in word_dict.iteritems():
        tf_dict[word] = count/float(bow_count)

    return tf_dict

def compute_IDF(doc_list):
    import math
    idf_dict = {}
    N = len(doc_list)

    #count number of documents that contain this word
    idf_dict = dict.fromkeys(doc_list[0].keys(), 0)
    for doc in doc_list:
        for word, count in doc.items():
            if count > 0:
                idf_dict[word] += 1

    for word, count in idf_dict.items():
        idf_dict[word] = math.log(N/float(count))

    return idf_dict

def compute_TFIDF(tf_bow, idfs):
    tfidf = {}
    for word, val in tf_bow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

# getting bag of words as sample
def bow(feature, kmeans):
    unique, counts = np.unique(kmeans.predict(feature), return_counts=True)
    no_of_centers = kmeans.cluster_centers_.shape[0]
    sample = [0]*no_of_centers
    for j in range(no_of_centers):
        if j in unique:
            sample[j] = counts[np.where(unique == j)][0]
    return sample

def boaw(feature, codebook, num_assign=10):
    # Faster for large dataset
    x_idx = cKDTree(codebook).query(feature, k=num_assign)[1].reshape(-1)

    word_dict = dict.fromkeys([j for j in range(len(codebook))], 0)
    for word in x_idx:
        word_dict[word] += 1

    boaw = np.array([*word_dict.values()])
    return boaw, word_dict

def generate_boaw(X, codebook):
    boaws = []
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    with tqdm(total=len(X)) as t:
        for x_boaw, word_dict in pool.imap(partial(boaw, codebook=codebook), X):
            tf_dict = {}
            for word, count in word_dict.items():
                tf_dict[word] = count
            boaws.append(x_boaw*np.log(np.array([*tf_dict.values()])))
            t.update(1)
    pool.close()
    pool.join()
    return boaws

def evaluate(ensem_preds, targets):
    """
      Evaluate the prediction by providing metrics & also the best threshold (to get the highest f1-score)
      Ex: AUC, Accurary, Precision, Recall, F1-Score.
      Then print these metrics
      Args:
        - ensem_preds: predictions for ids 
                       :type: a numpy array
        - targets: the actual results of ids 
                   :type: a numpy array                 
      Returns:
        - None                  
    """     
    best_th = 0
    best_score = 0

    for th in np.arange(0.0, 0.6, 0.01):
        pred = (ensem_preds > th).astype(int)
        score = f1_score(targets, pred)
        if score > best_score:
            best_th = th
            best_score = score

    print(f"\nAUC score: {roc_auc_score(targets, ensem_preds):12.4f}")
    print(f"Best threshold {best_th:12.4f}")

    preds = (ensem_preds > best_th).astype(int)
    # print(classification_report(targets, preds, digits=3))

    cm1 = confusion_matrix(targets, preds)
    print('\nConfusion Matrix : \n', cm1)
    precision, recall, f1 = compute_metrics(cm1)
    
    print('\n=============')
    print (f'Precision    : {precision:12.4f}')
    
    print(f'Recall : {recall:12.4f}')
    
    print(f'F1 Score : {f1:12.4f}')
    
    total1=sum(sum(cm1))

    print('\n=============')
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print (f'Accuracy    : {accuracy1:12.4f}')

def get_model(pos_scale, c=1):
    if Classifier == 'SVM':
        model = LinearSVC(C=c, class_weight='balanced', random_state=seed)
    else:
        model = xgb.XGBClassifier(max_depth=6, learning_rate=0.07,
            scale_pos_weight=pos_scale,
            n_estimators=200,
            subsample=1,
            colsample_bytree=1,
            eta=1, objective='binary:logistic',
            eval_metric='auc'
        )
    return model

# Train COVID-19

In [20]:
 if not os.path.exists(OUTPUT_DIR + 'df_label_covid_5fold.csv'):
    folds = df.copy()
    Fold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for n, (train_index, val_index) in enumerate(Fold.split(folds, folds['label_covid'])):
        folds.loc[val_index, 'fold'] = int(n)
    folds['fold'] = folds['fold'].astype(int)
    folds.to_csv(OUTPUT_DIR + 'df_label_covid_5fold.csv', index=False)
else:
    folds = pd.read_csv(OUTPUT_DIR + 'df_label_covid_5fold.csv')

In [21]:
y = folds['label_covid']
pos_scale = (y == 0).sum() / (y == 1).sum()
print(pos_scale)
targets = []
preds = []
aucs = []

for fold in range(5):
    train_idx = folds['fold'] != fold
    valid_idx = folds['fold'] == fold
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_val = X[valid_idx]
    y_val = y[valid_idx]

    targets.append(y_val)
    
    if PRETRAIN == 'OpenSmileBoAW':
        from sklearn.cluster import MiniBatchKMeans

        p = int(X_train.shape[1]/130)
        X_train1 = X_train[:, :(p-1)*65].reshape(-1, p-1, 65)
        X_train2 = X_train[:, (p-1)*65:].reshape(-1, (p+1), 65)
        X_val1 = X_val[:, :(p-1)*65].reshape(-1, (p-1), 65)
        X_val2 = X_val[:, (p-1)*65:].reshape(-1, (p+1), 65)

        t = time.time()
        vocab1 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train1.reshape(-1, 65))
        codebook1 = vocab1.cluster_centers_
        print("Kmean =", time.time()-t)
        X_train1 = generate_boaw(X_train1, codebook1)
        X_val1 = generate_boaw(X_val1, codebook1)
        
        vocab2 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train2.reshape(-1, 65))
        codebook2 = vocab2.cluster_centers_
        X_train2 = generate_boaw(X_train2, codebook2)
        X_val2 = generate_boaw(X_val2, codebook2)

        X_train = np.nan_to_num(np.concatenate([X_train1, X_train2], axis=1))  # [num_sample, codebook_size*2]
        X_val = np.nan_to_num(np.concatenate([X_val1, X_val2], axis=1))

    model = get_model(pos_scale=pos_scale)
    model.fit(X_train, y_train)

    pred = model.predict(X_val)
    preds.append(pred)
    auc = roc_auc_score(y_val, pred)
    aucs.append(auc)
    del model

targets = np.concatenate(targets)
preds = np.concatenate(preds)

print("(!) cv5 AUC ", np.mean(aucs), np.std(aucs))
evaluate(preds, targets)

2.786127167630058
0.7471726190476189
0.8586393331831494
0.8021701584440941
0.8539460839528423
0.8228955470451302
(!) cv5 AUC  0.816964748334567 0.04056941158112057

AUC score:       0.8168
Best threshold       0.0000

Confusion Matrix : 
 [[803 161]
 [ 69 277]]

Precision    :       0.6324
Recall :       0.8006
F1 Score :       0.7066

Accuracy    :       0.8244


# Train abnormal

In [22]:
 if not os.path.exists(OUTPUT_DIR + 'df_label_abnormal_5fold.csv'):
    folds = df.copy()
    Fold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for n, (train_index, val_index) in enumerate(Fold.split(folds, folds['label_abnormal'])):
        folds.loc[val_index, 'fold'] = int(n)
    folds['fold'] = folds['fold'].astype(int)
    folds.to_csv(OUTPUT_DIR + 'df_label_abnormal_5fold.csv', index=False)
else:
    folds = pd.read_csv(OUTPUT_DIR + 'df_label_abnormal_5fold.csv')

In [23]:
y = folds['label_abnormal']
pos_scale = (y == 0).sum() / (y == 1).sum()
print(pos_scale)
targets = []
preds = []
aucs = []

def get_model(pos_scale):
    model = xgb.XGBClassifier(
        max_depth=7,
        scale_pos_weight=pos_scale,
        learning_rate=0.3,
        n_estimators=200,
        subsample=1,
        colsample_bytree=1,
        nthread=-1,
        seed=seed,
        eval_metric='logloss'
    )
    return model

for fold in range(5):
    train_idx = folds['fold'] != fold
    valid_idx = folds['fold'] == fold
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_val = X[valid_idx]
    y_val = y[valid_idx]

    targets.append(y_val)
    
    if PRETRAIN == 'OpenSmileBoAW':
        from sklearn.cluster import MiniBatchKMeans

        p = int(X_train.shape[1]/130)
        X_train1 = X_train[:, :(p-1)*65].reshape(-1, p-1, 65)
        X_train2 = X_train[:, (p-1)*65:].reshape(-1, (p+1), 65)
        X_val1 = X_val[:, :(p-1)*65].reshape(-1, (p-1), 65)
        X_val2 = X_val[:, (p-1)*65:].reshape(-1, (p+1), 65)

        t = time.time()
        vocab1 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train1.reshape(-1, 65))
        codebook1 = vocab1.cluster_centers_
        print("Kmean =", time.time()-t)
        X_train1 = generate_boaw(X_train1, codebook1)
        X_val1 = generate_boaw(X_val1, codebook1)
        
        vocab2 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train2.reshape(-1, 65))
        codebook2 = vocab2.cluster_centers_
        X_train2 = generate_boaw(X_train2, codebook2)
        X_val2 = generate_boaw(X_val2, codebook2)

        X_train = np.nan_to_num(np.concatenate([X_train1, X_train2], axis=1))  # [num_sample, codebook_size*2]
        X_val = np.nan_to_num(np.concatenate([X_val1, X_val2], axis=1))

    model = get_model(pos_scale=pos_scale)

    pred = model.predict(X_val)
    preds.append(pred)
    auc = roc_auc_score(y_val, pred)
    aucs.append(auc)
    print(auc)
    del model

targets = np.concatenate(targets)
preds = np.concatenate(preds)

print("(!) cv5 AUC ", np.mean(aucs), np.std(aucs))
evaluate(preds, targets)

1.7010309278350515
0.7823180256169946
0.7259293970634177
0.7965635738831615
0.7219931271477663
0.723805060918463
(!) cv5 AUC  0.7501218369259606 0.032442258421842146

AUC score:       0.7501
Best threshold       0.0000

Confusion Matrix : 
 [[741  84]
 [193 292]]

Precision    :       0.7766
Recall :       0.6021
F1 Score :       0.6783

Accuracy    :       0.7885


## Test other model

### IsolationForest
Isolation Forest use Sklearn library

In [26]:
y = folds['label_abnormal']
pos_scale = (y == 0).sum() / (y == 1).sum()
print(pos_scale)

def get_model(pos_scale):
    model = IsolationForest(n_estimators=500, max_samples='auto', contamination=0.1, n_jobs=-1, random_state=seed) 
    return model

1.7010309278350515


In [27]:
targets = []
preds = []
aucs = []

for fold in range(5):
    train_idx = folds['fold'] != fold
    valid_idx = folds['fold'] == fold
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_val = X[valid_idx]
    y_val = y[valid_idx]

    targets.append(y_val)
    
    if PRETRAIN == 'OpenSmileBoAW':
        from sklearn.cluster import MiniBatchKMeans

        p = int(X_train.shape[1]/130)
        X_train1 = X_train[:, :(p-1)*65].reshape(-1, p-1, 65)
        X_train2 = X_train[:, (p-1)*65:].reshape(-1, (p+1), 65)
        X_val1 = X_val[:, :(p-1)*65].reshape(-1, (p-1), 65)
        X_val2 = X_val[:, (p-1)*65:].reshape(-1, (p+1), 65)

        t = time.time()
        vocab1 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train1.reshape(-1, 65))
        codebook1 = vocab1.cluster_centers_
        print("Kmean =", time.time()-t)
        X_train1 = generate_boaw(X_train1, codebook1)
        X_val1 = generate_boaw(X_val1, codebook1)
        
        vocab2 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train2.reshape(-1, 65))
        codebook2 = vocab2.cluster_centers_
        X_train2 = generate_boaw(X_train2, codebook2)
        X_val2 = generate_boaw(X_val2, codebook2)

        X_train = np.nan_to_num(np.concatenate([X_train1, X_train2], axis=1))  # [num_sample, codebook_size*2]
        X_val = np.nan_to_num(np.concatenate([X_val1, X_val2], axis=1))

    model = get_model(pos_scale)
    model.fit(X_train)

    scores = (-1.0) * model.decision_function(X_val)
    pred = scores.flatten()
    preds.append(pred)
    auc = roc_auc_score(y_val, pred)
    aucs.append(auc)
    print(auc)
    del model

targets = np.concatenate(targets)
preds = np.concatenate(preds)

print("(!) cv5 AUC ", np.mean(aucs), np.std(aucs))
evaluate(preds, targets)

0.4850359262730396
0.544704779756326
0.5323961262105591
0.5684473601999376
0.5526398000624805
(!) cv5 AUC  0.5366447985004685 0.02833278388113743

AUC score:       0.5352
Best threshold       0.0000

Confusion Matrix : 
 [[740  85]
 [437  48]]

Precision    :       0.3609
Recall :       0.0990
F1 Score :       0.1553

Accuracy    :       0.6015


### XGBOD

In [28]:
y = folds['label_abnormal']
pos_scale = (y == 0).sum() / (y == 1).sum()
print(pos_scale)

from pyod.models.xgbod import XGBOD

def get_model(pos_scale):
    # Model candidate
    model = XGBOD(max_depth=7,
        scale_pos_weight=pos_scale,
        learning_rate=0.3,
        n_estimators=200,
        subsample=1,
        colsample_bytree=1,
        nthread=-1,
        seed=seed,
        eval_metric='logloss')
    return model

1.7010309278350515


In [29]:
targets = []
preds = []
aucs = []

for fold in range(5):
    train_idx = folds['fold'] != fold
    valid_idx = folds['fold'] == fold
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_val = X[valid_idx]
    y_val = y[valid_idx]

    targets.append(y_val)
    
    if PRETRAIN == 'OpenSmileBoAW':
        from sklearn.cluster import MiniBatchKMeans

        p = int(X_train.shape[1]/130)
        X_train1 = X_train[:, :(p-1)*65].reshape(-1, p-1, 65)
        X_train2 = X_train[:, (p-1)*65:].reshape(-1, (p+1), 65)
        X_val1 = X_val[:, :(p-1)*65].reshape(-1, (p-1), 65)
        X_val2 = X_val[:, (p-1)*65:].reshape(-1, (p+1), 65)

        t = time.time()
        vocab1 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train1.reshape(-1, 65))
        codebook1 = vocab1.cluster_centers_
        print("Kmean =", time.time()-t)
        X_train1 = generate_boaw(X_train1, codebook1)
        X_val1 = generate_boaw(X_val1, codebook1)
        
        vocab2 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train2.reshape(-1, 65))
        codebook2 = vocab2.cluster_centers_
        X_train2 = generate_boaw(X_train2, codebook2)
        X_val2 = generate_boaw(X_val2, codebook2)

        X_train = np.nan_to_num(np.concatenate([X_train1, X_train2], axis=1))  # [num_sample, codebook_size*2]
        X_val = np.nan_to_num(np.concatenate([X_val1, X_val2], axis=1))

    model = get_model(pos_scale)
    model.fit(X_train, y_train)

    pred = model.decision_function(X_val)  # predict raw outlier scores on test
    auc = roc_auc_score(y_val, pred)
    preds.append(pred)
    aucs.append(auc)
    print(auc)
    del model

targets = np.concatenate(targets)
preds = np.concatenate(preds)

print("(!) cv5 AUC ", np.mean(aucs), np.std(aucs))
evaluate(preds, targets)

[01:00:05] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.8647922524211183
[01:05:11] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.8091221493283349
[01:10:14] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find

### IForest
Isolation Forest use pyod library

In [30]:
y = folds['label_abnormal']
pos_scale = (y == 0).sum() / (y == 1).sum()
print(pos_scale)

from pyod.models.iforest import IForest

def get_model(pos_scale):
    model = IForest(n_estimators=500, max_samples='auto', contamination=0.1, n_jobs=-1, random_state=seed) 
    return model

1.7010309278350515


In [31]:
targets = []
preds = []
aucs = []

for fold in range(5):
    train_idx = folds['fold'] != fold
    valid_idx = folds['fold'] == fold
    X_train = X[train_idx]
    y_train = y[train_idx]
    X_val = X[valid_idx]
    y_val = y[valid_idx]

    targets.append(y_val)
    
    if PRETRAIN == 'OpenSmileBoAW':
        from sklearn.cluster import MiniBatchKMeans

        p = int(X_train.shape[1]/130)
        X_train1 = X_train[:, :(p-1)*65].reshape(-1, p-1, 65)
        X_train2 = X_train[:, (p-1)*65:].reshape(-1, (p+1), 65)
        X_val1 = X_val[:, :(p-1)*65].reshape(-1, (p-1), 65)
        X_val2 = X_val[:, (p-1)*65:].reshape(-1, (p+1), 65)

        t = time.time()
        vocab1 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train1.reshape(-1, 65))
        codebook1 = vocab1.cluster_centers_
        print("Kmean =", time.time()-t)
        X_train1 = generate_boaw(X_train1, codebook1)
        X_val1 = generate_boaw(X_val1, codebook1)
        
        vocab2 = MiniBatchKMeans(n_clusters=codebook_size).fit(X_train2.reshape(-1, 65))
        codebook2 = vocab2.cluster_centers_
        X_train2 = generate_boaw(X_train2, codebook2)
        X_val2 = generate_boaw(X_val2, codebook2)

        X_train = np.nan_to_num(np.concatenate([X_train1, X_train2], axis=1))  # [num_sample, codebook_size*2]
        X_val = np.nan_to_num(np.concatenate([X_val1, X_val2], axis=1))

    model = get_model(pos_scale)
    model.fit(X_train)

    pred = model.decision_function(X_val)  # predict raw outlier scores on test
    auc = roc_auc_score(y_val, pred)
    preds.append(pred)
    aucs.append(auc)
    print(auc)
    del model

targets = np.concatenate(targets)
preds = np.concatenate(preds)

print("(!) cv5 AUC ", np.mean(aucs), np.std(aucs))
evaluate(preds, targets)

0.4850359262730396
0.544704779756326
0.5323961262105591
0.5684473601999376
0.5526398000624805
(!) cv5 AUC  0.5366447985004685 0.02833278388113743

AUC score:       0.5352
Best threshold       0.0000

Confusion Matrix : 
 [[740  85]
 [437  48]]

Precision    :       0.3609
Recall :       0.0990
F1 Score :       0.1553

Accuracy    :       0.6015


# Save notebook

In [32]:
!cp "SoundDr_cough.ipynb" "$OUTPUT_DIR/SoundDr_cough_frill_svm_2022.ipynb"